<b>Dieses Notebook enthält den Scrapper </b>\
Es kann bis zum ersten aufruf von scrape() problemlos ausgeführt werden

In [118]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
import typing
import pickle
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
import random
import json
import re
import urllib
import base64

In [75]:
img_src="https://images.thalia.media/00/-/81e43ca44c6f4e0b9de88c84f5257a6e/bleach-taschenbuch-tite-kubo.jpeg"


In [108]:
html_source = "No"
url="https://www.worldometers.info/world-population/population-by-country/"
url="https://www.thalia.de/shop/home/artikeldetails/A1000847310"  #Bleach
url="https://www.thalia.de/shop/home/artikeldetails/A1063615999"  #Star Wars
url="https://www.thalia.de/shop/home/artikeldetails/A1057771737"  #g buch

In [109]:
def printInfo(browser:WebDriver):
    print(browser.title)
    print(browser.current_url)
    driver_ua = browser.execute_script("return navigator.userAgent")
    print("User agent:",driver_ua)

def clickDetailsBut(browser:WebDriver):
    #detail_but= browser.find_element(By.CSS_SELECTOR,"button[data-dialog='details']")
    #click on detail button with js
    outerButHTML_str:str = browser.execute_script(r"let detailbut=document.querySelector('button[data-dialog=\'details\']') ;console.log(detailbut); detailbut.click(); detailbut.outerHTML")
    print(outerButHTML_str)
def declineCookies(browser:WebDriver):
    #browser.execute_script(r"let cookie=document.querySelector('#usercentrics-root');let but=cookie.shadowRoot.querySelector('button[data-testid=\"uc-deny-all-button\"]').click();return but.outerHTML")
    browser.execute_script(r"try{ let cookie=document.querySelector('#usercentrics-root');let but=cookie.shadowRoot.querySelector('button[data-testid=\"uc-deny-all-button\"]').click();return but.outerHTML}catch(e){console.log('error at accepting cookies ;ignore it')}")



In [110]:
my_user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
my_user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0"
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless=false")

# Set the custom User-Agent
#chrome_options.add_argument(f"--user-agent={my_user_agent}")
chrome_options.enable_downloads=True
chrome_options.arguments
url

'https://www.thalia.de/shop/home/artikeldetails/A1057771737'

In [111]:
def downloadImage(browser,saveName="savedImg"):
    save_script=R''' try{
    let src=document.querySelector("image-zoom picture source, tab-panel picture source").srcset
    function urlContentToDataUri(url){
    return  fetch(url)
            .then( response => response.blob() )
            .then( blob => new Promise( callback =>{
                let reader = new FileReader() ;
                reader.onload = function(){ callback(this.result) } ;
                reader.readAsDataURL(blob) ;
            }) ) ;
}
    let base64=await urlContentToDataUri(src);
    return base64.substring(21);;
    }
    catch(e){return ""}
''' 
    print(f"exe script {save_script}")
    base_in_64=browser.execute_script(save_script)
    print(base_in_64[0:400])
    print("ende save img")

In [125]:
def saveBase64Img(base_input_64="",dst="img.png"):
    img = base64.b64decode(base_input_64)
    with open(dst, 'wb') as f:
        f.write(img)


In [112]:
# nur ein test ob Selenium tatsächlic läuft
def testSelenium():
    # dies Zeile benötigt einen Chrome driver der in env-var bekannt ist
    # alternativ:   driver = webdriver.Chrome('./chromedriver') <- wenn driver in gleichen Ordner ist
    #try:
    browser= webdriver.Chrome()
    browser.get(url)

    html_source= browser.page_source
    printInfo(browser)

    sleep(5)

    #decline cookies
    declineCookies(browser)
    sleep(5)
    
    #clickDetailsBut(browser)
    downloadImage(browser,"Test")
    ''' sleep(5)
    browser.execute_script('window.location="https://www.thalia.de/shop/home/artikeldetails/A1020323321"')
    sleep(3)
    next_html=browser.page_source

    sleep(5)
    browser.execute_script('window.location="https://www.thalia.de/shop/home/artikeldetails/A1020881790"')
    sleep(3)
    next_html_pp=browser.page_source
'''
    sleep(10)
    browser.close()
    #except Exception as e:
    #  print("Webdriver preumably closed",e)
testSelenium()

'Späte Erlösung' von 'Hugo Stamm' - Buch - '978-3-347-03047-3'
https://www.thalia.de/shop/home/artikeldetails/A1057771737
User agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36
exe script  
    let src=document.querySelector("image-zoom picture source, tab-panel picture source").srcset
    function urlContentToDataUri(url){
    return  fetch(url)
            .then( response => response.blob() )
            .then( blob => new Promise( callback =>{
                let reader = new FileReader() ;
                reader.onload = function(){ callback(this.result) } ;
                reader.readAsDataURL(blob) ;
            }) ) ;
}
    let base64=await urlContentToDataUri(src);
    return base64;


ende save img


In [6]:
html_source

'No'

In [7]:
soup = BeautifulSoup(html_source, "html.parser") 

In [86]:
soup.select(".kurzbeschreibung")[0:3]

[]

In [71]:
#testobj={"Titel": "Wilhelm Brinkmeyers Abenteuer von ihm selbst erzählt", "Autor": ["Rudolf Huch"], "KurzBesch": "", "ExtraInfo": "Buch (Taschenbuch)", "Kategorien": "StartseiteGesc//Bücher/Romane&Erzählungenenglisch/Literatur", "Beschreibung": "Eheu! Mit diesem Weheruf aus dem klassischen Altertum pflegte ich während meiner Gymnasialzeit das göttliche Dreigespann des Phaeton zu begrüßen, jenes verwegenen Jünglings, der sich, wie uns Ovid überliefert, dereinst der Lenkung des Sonnenwagens unterwunden hat, aber weil ihm die Kräfte des Apollon fehlten, auf das elendste dabei umgekommen ist. Eine Erzählung, die mir in dem derzeitigen philosophischen Abschnitte meines Lebens höchst bedeutungsvoll erscheint, als ein Symbolum der allermeisten menschlichen Bestrebungen, ja vielleicht des menschlichen Lebens überhaupt. Wenn wir nämlich das von jenem Phaeton befahrene Himmelsgewölbe dem Tummelplatze dieser Erde und sein Gespann unserm Schicksal gleichsetzen, so ergibt sich, daß wir uns wohl alle in grüner Jugend vermessen, das Gefährte nach unserm Willen über die Bahn zu lenken, da wir doch nach einiger Strecke erkennen müssen, sofern wir nicht bis zum Ende in geistiger Blindheit beharren, daß vielmehr das Gespann uns führt, wohin es ihm beliebt. Auch hinsichtlich der Gleichsetzung unsers Schicksals mit einem durchgehenden Gespann erscheint mir der Vergleich keineswegs zu hinken. Irgendwas von Sinn und Verstand vermag ich in dem Walten der Schicksalsmacht wenigstens in meinem Leben nicht zu entdecken. Was nun schließlich das klägliche Ende des Unternehmers anlangt, so ist, solange die Welt steht, noch keines Menschen Fahrt anders ausgegangen als mit dem Tode. ¿ Der aufmerkende Leser wird nicht erst der ausdrücklichen Versicherung bedürfen, daß mein Weheruf nicht etwa einer Abneigung wider die Wissenschaften entsprang. Das war so wenig der Fall, daß ich schon von der Obertertia an beflissen war, mich auf mein erwähltes Fach, Theologie und Philologie, vorzubereiten, indem ich Sextanern und Quintanern, deren Eltern mich darum anließen, gegen ein geringes Taschengeld Nachhilfestunden erteilte.", "Rezension": "", "Einband": "Taschenbuch", "Erscheinungsdatum": "09.01.2023", "Verlag": "Culturea", "Seitenzahl": "204", "Maße (L/B/H)": "22/17/1,2 cm", "Gewicht": "326 g", "Sprache": "Deutsch", "ISBN": "979-10-419-0291-0", "Img": ["https://images.thalia.media/00/-/adcf6c47e8474d2198f3dca177c75c6e/wilhelm-brinkmeyers-abenteuer-von-ihm-selbst-erzaehlt-taschenbuch-rudolf-huch.jpeg"], "url": "https://www.thalia.de/shop/home/artikeldetails/A1067788334"}
def testIsGermanBook(details)->bool:
    book_reg="fremd|Musik|kalendar|schreibwaren|Geschenk|Spielwaren|Spiel|eReader|Zubehör|Englisch"
    x=re.search(book_reg,details['ExtraInfo']+details['Kategorien'],flags=re.IGNORECASE)
    return (True if x==None else  False)
#testIsGermanBook(testobj)

<b>Die Methode die das DOM/Soup einer Webseite nimmt und die wichtigen Infos extrahiert 

In [9]:
def extract(DOM)->dict:
        
    details={}
    
    title_el= DOM.select_one ("section.basis-informationen h1")
    details["Titel"] = title_el.text if title_el != None else ""

    autors_el_list= DOM.select("a.autor-name")
    details["Autor"] = [autor.text for autor in autors_el_list ]

    short_Desc_el= DOM.select_one ("section.basis-informationen .untertitel")
    details["KurzBesch"] = short_Desc_el.text if short_Desc_el != None else ""

    extra_Info_el= DOM.select_one ("section.basis-informationen ul+ p")
    details["ExtraInfo"] = extra_Info_el.text if extra_Info_el != None else ""

    categories_el= DOM.select_one ("ul.breadcrumb-list")
    details["Kategorien"] ="".join(  [cat.strip() for cat in categories_el.text.split()] ) if categories_el != None else ""


    desc_el= DOM.select_one ('dialog[data-dialog-name="zusatztexte"] section.zusatztext div')
    if desc_el ==None: desc_el= DOM.select_one ('.inhalt-beschreibung .kurzbeschreibung')
    details["Beschreibung"] = desc_el.text.strip() if desc_el != None else ""
    
    review_el= DOM.select_one('dialog[data-dialog-name="zusatztexte"] section.zusatztext-gruppe p')
    details["Rezension"] = review_el.text if review_el != None else ""

    detail_dialog_div= DOM.select_one("dialog[data-dialog-name='details'] div.artikeldetails ")
    for el in  detail_dialog_div.find_all("section") if detail_dialog_div!=None else [] :
            dia_key_el=el.find('h3', {'class': 'detailbezeichnung'})
            val_el=el.select_one(".single-value, a")
            if dia_key_el !=None  and val_el != None:
                details[dia_key_el.text.strip() ]=val_el.text

    img_el_list= DOM.select("image-zoom picture source, tab-panel picture source")
    details["Img"] = [img['srcset'] for img in img_el_list ]
    
    #print(details)
    return details
data=extract(soup)    

In [10]:

def save(obj:dict[str,str] ,path:str="Data/book_data.1.0"):
    if obj==None: return
    with open(path, 'a',encoding= "utf-8") as file:
        json.dump(obj, file,ensure_ascii=False)
        file.write("\n")
#save(data)

In [11]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))
#pretty(data)

In [12]:
def loadURLs(path:str="TestUrl/1000_test_Urls.txt"):
    with open(path, 'r',encoding= "utf-8") as file:
        contents = file.read()
        return contents.split("\n") if contents!=None else []

In [13]:
urlList=loadURLs()
print(len( urlList),urlList[0:10],"...",urlList[-10:],'\n',random.choice( urlList) )

1152 ['https://www.thalia.de/shop/home/artikeldetails/A1066698677', 'https://www.thalia.de/shop/home/artikeldetails/A1047574159', 'https://www.thalia.de/shop/home/artikeldetails/A1069776105', 'https://www.thalia.de/shop/home/artikeldetails/A1063536656', 'https://www.thalia.de/shop/home/artikeldetails/A1050290421', 'https://www.thalia.de/shop/home/artikeldetails/A1050354428', 'https://www.thalia.de/shop/home/artikeldetails/A1069945964', 'https://www.thalia.de/shop/home/artikeldetails/A1068344029', 'https://www.thalia.de/shop/home/artikeldetails/A1054221230', 'https://www.thalia.de/shop/home/artikeldetails/A1069888647'] ... ['https://www.thalia.de/shop/home/artikeldetails/A1067383475', 'https://www.thalia.de/shop/home/artikeldetails/A1067383360', 'https://www.thalia.de/shop/home/artikeldetails/A1067372013', 'https://www.thalia.de/shop/home/artikeldetails/A1067361992', 'https://www.thalia.de/shop/home/artikeldetails/A1067334401', 'https://www.thalia.de/shop/home/artikeldetails/A1067334306

In [14]:
def dragonMsg(sep:str="|#|"):
    print( sep.join([f'\x1b[{90+(x%8)}m\U0001f409\ufe0e' for x in range(0,10)]) ,"\n\n"+3*"\t"+"\x1b[3;37m| (OvO) |" )

Die Scrape Methode\
sie nimmt ein File mit '\n' separierten Buch-artikel URLS und (!)appendet die Daten in den spezifizierten out_file 

In [15]:
def scrape(url_path:str,output_file:str):
    try:
        scrape_urls = loadURLs(url_path)
        print(f"urls length {len(scrape_urls)} {scrape_urls[0:5]} ...{scrape_urls[-5:]}")
    except Exception as e:
        print(e,"something went wrong aborted")
        return

    cur_url_index=0
    browser:WebDriver= webdriver.Chrome()
    browser.get(scrape_urls[0])
    sleep(1)
    declineCookies(browser)
    sleep(3)

    error_counter=0
    for current_url in scrape_urls:
        try:
            if(current_url== ""):cur_url_index+=1;continue
            if(cur_url_index!=0):
                #!!! important
                sleep(4+ 1*random.random())
                browser.execute_script(f'window.location="{current_url}"')
                sleep(1)
            
            cur_html= browser.page_source
            cur_DOM=BeautifulSoup(cur_html,"html.parser")
            extr_dat=extract(cur_DOM)
            extr_dat["url"]=current_url
            #print("Page title",browser.title,"\n",extr_dat)
            save(extr_dat,output_file)
        except:
            error_counter+=1
            print(f"something went wrong at {current_url}-- skiped to next err_count:{error_counter}")  
            if error_counter>10: print("abour err_count to high"); return
            
        cur_url_index+=1
      
    browser.close()
    dragonMsg()
    print("finshed succesfully\n")    

<b>Das startet einen Browser und beginnt das scrapen\
Nur ausführen wenn gewollt!\
<i>Das ist ein kleiner Test mit nur 10 URLs im file

In [16]:
scrape("TestURL/10_test_Urls.txt","Data/10_test_Urls_data.jsonl" )

urls length 11 ['https://www.thalia.de/shop/home/artikeldetails/A1067810523', 'https://www.thalia.de/shop/home/artikeldetails/A1067806323', 'https://www.thalia.de/shop/home/artikeldetails/A1067811159', 'https://www.thalia.de/shop/home/artikeldetails/A1067800008', 'https://www.thalia.de/shop/home/artikeldetails/A1067795103'] ...['https://www.thalia.de/shop/home/artikeldetails/A1067792440', 'https://www.thalia.de/shop/home/artikeldetails/A1067787314', 'https://www.thalia.de/shop/home/artikeldetails/A1067787285', 'https://www.thalia.de/shop/home/artikeldetails/A1067788334', '']
🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎ 

			| (OvO) |
finshed succesfully



<b> Das braucht mindesten 2h!

In [16]:
scrape("TestURL/1000_test_Urls.txt","Data/1000_test_Urls_data.txt" )

urls length 1152 ['https://www.thalia.de/shop/home/artikeldetails/A1066698677', 'https://www.thalia.de/shop/home/artikeldetails/A1047574159', 'https://www.thalia.de/shop/home/artikeldetails/A1069776105', 'https://www.thalia.de/shop/home/artikeldetails/A1063536656', 'https://www.thalia.de/shop/home/artikeldetails/A1050290421'] ...['https://www.thalia.de/shop/home/artikeldetails/A1067334306', 'https://www.thalia.de/shop/home/artikeldetails/A1067333653', 'https://www.thalia.de/shop/home/artikeldetails/A1067333754', 'https://www.thalia.de/shop/home/artikeldetails/A1067333711', 'https://www.thalia.de/shop/home/artikeldetails/A1067333835']
🐉︎🐉︎🐉︎🐉︎🐉︎🐉︎🐉︎🐉︎🐉︎🐉︎ 

			| (OvO) |
finshed succesfully



Diese Methode erstellt ein URL file von Buch-artikelseiten\
die von ihr erstellten files können für die scrape Methode verwendet werden\
Sie braucht entweder einen categorie URL oder einen such URL.
<li> alle category urls kann  von thalia sich geben mit https://www.thalia.de/sitemaps/kategorie/kategorie.xml \
(ist schon extrahiert in Ordner Test/URL/categorieurl.txt)
<li>Oder man geht auf die thalia Webseite und sucht irgendetwas und kopiert sich den url raus.\
Die Suche hat unten einen [weitere laden] Knopf, dieser wird solange gerückt bis alle geladen sind oder eine Obergrenze erreicht ist. Dann werden all Verweise der Seite auf Buchseiten extrahiert und in ein FIle geschrieben

In [55]:
# goes to a kategorie site and clicks the "weitere laden button until no more remaining click are there
# and saves all link of saerch results to a file
def collect_urls(search_url:str,output_file:str="default"):
    browser:WebDriver= webdriver.Chrome()
    browser.get(search_url)
    sleep(1)
    declineCookies(browser)
    sleep(1)
    cat_urls= browser.execute_script(r''' 
let $=document.querySelector.bind(document);
let $A=document.querySelectorAll.bind(document);
const sleep= async(m_time)=> await new Promise(resolve => setTimeout(resolve,1000*m_time));
let weiter_laden=$("button[interaction=weitere-ergebnisse-laden]")
weiter_laden.click();
await sleep(3);
weiter_laden.click();
await sleep(3);
let amount=$(".element-text-standard-strong.ergebnisanzeige")
let amountsar=   amount.innerText.split(/\D+/).slice(0,2).map(x=>+x)
let remaining=amountsar[1]-amountsar[0];
let max_click=100;
while(max_click>0&&remaining>0){
           max_click--;     
weiter_laden.click();                      
await sleep(1);
amountsar=   amount.innerText.split(/\D+/).slice(0,2).map(x=>+x)
remaining=amountsar[1]-amountsar[0];
}                                                                            
let all_links=[...$A("a[caption=suchergebnis-klick]")].map(x=>x.href)  
return    [...new Set(all_links)].join("\n")                                                                      
''')
    print(cat_urls)
    used_out_file="TestUrl/"
    if output_file=="default":
        category=search_url.split("/")[-1]
        if category=="":category=  search_url.split("/")[-2]
        used_out_file+=category+".txt"
    else:
        used_out_file=output_file
    print(f"found urls were written to {used_out_file}")
    with open(used_out_file, 'w',encoding= "utf-8") as file:
        file.write(cat_urls)
    
    #browser.close()
    sleep(10)
    dragonMsg()
    print("finshed succesfully\n")  

Führt es aus

In [58]:
collect_urls("https://www.thalia.de/kategorie/feste-partys-9422/")

In [59]:
scrape("TestURL/feste-partys-9422.txt","Data/feste-partys-9422.jsonl" )

urls length 309 ['https://www.thalia.de/shop/home/artikeldetails/A1070043630', 'https://www.thalia.de/shop/home/artikeldetails/A1059167496', 'https://www.thalia.de/shop/home/artikeldetails/A1062298141', 'https://www.thalia.de/shop/home/artikeldetails/A1042663896', 'https://www.thalia.de/shop/home/artikeldetails/A1071044009'] ...['https://www.thalia.de/shop/home/artikeldetails/A1058207959', 'https://www.thalia.de/shop/home/artikeldetails/A1059326648', 'https://www.thalia.de/shop/home/artikeldetails/A1058306854', 'https://www.thalia.de/shop/home/artikeldetails/A1058264123', 'https://www.thalia.de/shop/home/artikeldetails/A1058230338']
🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎|#|🐉︎ 

			| (OvO) |
finshed succesfully

